
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file


In [37]:
import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma

from bids import BIDSLayout
import mne
import os
import numpy as np

### grab the data and set up variables

In [19]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)







print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
sub = "D0057" #just do one at a time for now, can implement all in a for loop later
print(subjects)


filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                           extension='.edf', desc='clean', preload=False) #get line-noise filtered data



save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)



# # what to call the folder, is there a convention? Nahhh.

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivativ

c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


### Okay now actually do filtering

In [23]:
## Crop raw data to minimize processing time
new = crop_empty_data(filt, )

# Mark channel outliers as bad
# new.info['bads'] = channel_outlier_marker(new, 4)
# Exclude bad channels
good = new.copy().drop_channels(filt.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
del new

Reading 0 ... 3219820  =      0.000 ...  1572.178 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.


In [31]:

## High Gamma Filter and epoching
out = []

trialType = "i75s25"

for epoch, t, name in zip(
        ("Stimulus"+trialType, "Stimulus"+trialType,  "Response"+trialType),
        ((-0.5, 0), (0, 1), (-0.5, 0.5)),
        ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, epoch, times, preload=True)
    outliers_to_nan(trials, outliers=10)
    gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(trials, "0.5s")
    trials.resample(100)
    trials.filenames = good.filenames
    out.append(trials)

print(out)

base = out.pop(0)
# ev1 = trial_ieeg(good, "baselineStimulusc75s25, ATT1,2", (-1, 2), "onset", (-1, 0.5))

# HG_ev1 = gamma.extract(ev1)
# crop_pad(HG_ev1, "500ms")

Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found


No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 3073 original time points ...
0 bad epochs dropped


100%|██████████| 23/23 [00:14<00:00,  1.63it/s]


Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 4097 original time points ...
0 bad epochs dropped


100%|██████████| 23/23 [00:18<00:00,  1.21it/s]


Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 4097 original time points ...
0 bad epochs dropped


100%|██████████| 23/23 [00:19<00:00,  1.19it/s]


[<Epochs |  23 events (all good), -0.5 – -0.01 s, baseline off, ~1.8 MB, data loaded,
 'Stimulusi75s25': 23>, <Epochs |  23 events (all good), 0 – 0.99 s, baseline off, ~3.3 MB, data loaded,
 'Stimulusi75s25': 23>, <Epochs |  23 events (all good), -0.5 – 0.49 s, baseline off, ~3.3 MB, data loaded,
 'Responsei75s25': 23>]


In [44]:
out[0].plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x800 with 4 Axes>

In [38]:
# this is to get rid of NaNs in the Epochs object so we can plot it. But there probably shouldn't be any NaNs in the first place...


# Assuming 'test' is your Epochs object
data = test.get_data()
data = np.nan_to_num(data, nan=0)
test = test.copy()  # Make a copy to preserve the original Epochs object
test._data = data   # Assign the modified data array back to the Epochs object



In [39]:
test.average().plot()

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


KeyboardInterrupt: 